Imports

In [1]:
import gymnasium as gym

import tensorflow as tf
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import mean_squared_error

Environment

In [2]:
env = gym.make("CartPole-v1")

Deep Q Learning Algo

- Train the model at each episode on a batch size of past experiences (replay buffer)
- At each training step, take few random past experiences from replay buffer (states, actions, reward, next_states, dones).
- It is convenient to pass only states rather than states-actions pair to model and get states-action pair from which we can take argmax for each state for optimal action.
- Eqn for Q_target : Q_target = r + gamma * max-over-actions (Q(next_state, actions)) -> this Q_target will be used for calculating loss with model's approximated Q value.
- Now calculate Q_target:
- Predict the next Q values using next states and take maximum Q value for each state. Put the values in eqn for Q_target.
- Calculate the approx Q values by passing states to model and calculate loss and gradients. Update the model vars by applying gradients.

- To take action in a environment use a epsilon-greedy-policy to both explore and exploit and perform a step in env. 
- Append these step experiences to a replay buffer so model can use them for training.
